# Pixel Drill for Indices outputs to CSV - Multi-site

| Authors:  | Bex Dunn|
|----------|----------------|
| Created: | March 6, 2019 |
| Last edited: | March 6, 2019 |

**Requirements:**

You need to run the following commands from the command line prior to launching jupyter notebooks from the same terminal so that the required libraries and paths are set:

`module use /g/data/v10/public/modules/modulefiles` 

`module load dea`

If you find an error or bug in this notebook, please either create an 'Issue' in the Github repository, or fix it yourself and create a 'Pull' request to contribute the updated notebook back into the repository (See the repository [README](https://github.com/GeoscienceAustralia/dea-notebooks/blob/master/README.rst) for instructions on creating a Pull request).

__Background:__ Data from the [Landsat](https://landsat.usgs.gov/about-landsat) 5,7 and 8 satellite missions are accessible through [Digital Earth Australia](http://www.ga.gov.au/about/projects/geographic/digital-earth-australia) (DEA).

__What does this notebook do?:__ This notebook takes a supplied CSV of site points. It runs a pixel drill through surface reflectance, calculates NDVI, Taselled cap wetness and greenness, and outputs a csv of values for each site and plots of each index for each site.

**Tags**: :index:`Landsat`,:index:`Landsat5`,:index:`Landsat7`,:index:`Landsat8`, :index:`pixeldrill`, :index:`DEAPlotting`, :index:`datacube.utils.geometry`, :index:`query`,:index:`Scripts`,:index:`tasseled_cap`, :index:`NDVI`,                                                                                                           :index:`DEADataHandling`, :index:`DEAPlotting`, :index:`load_clearlandsat`

import some modules

In [ ]:
import datacube
import datetime
import fiona
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio.mask
import rasterio.features
import shapely
import seaborn as sns
import sys
import xarray as xr

import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

from datacube.storage import masking
from datacube.utils import geometry

sys.path.append('../10_Scripts')
import DEADataHandling, DEAPlotting, TasseledCapTools, BandIndices

dc = datacube.Datacube(app='pixel drill')

%load_ext autoreload

%autoreload 2

#set up file to open 

In [18]:
inpath = '/g/data/r78/rjd547/Melbourne_water/150 points to check.csv'

### Convert csv latitude and longitude values into a geopandas geodatafrome

sites = pd.read_csv(inpath, delimiter=",", header=0,index_col=0)
#turn csv geometry into geopandas dataframe geometry using lambda functions and shapely
sites['geometry']=sites.apply(lambda z: shapely.geometry.Point(z.x, z.y), axis=1)
sites = gpd.GeoDataFrame(sites)
sites.head()

,x,y,geometry
109,144.208366,-37.722290,POINT (144.208366098071 -37.7222899289321)
69,144.444587,-37.758351,POINT (144.444586799121 -37.75835133303421)
123,144.600005,-37.554473,POINT (144.6000047609 -37.554472809368)
60,144.833754,-37.854078,POINT (144.833754427749 -37.8540777666739)
145,144.695334,-37.718812,POINT (144.695333814669 -37.7188115739301)


In [34]:
len(sites)

303

In [37]:
#define an output location
output_loc = '/g/data/r78/rjd547/Melbourne_water/'

### run this for multiple sites

for site in range(1,len(sites),1):
    print(site)    
    print(f'running for index {site}')

    query = {'lat':(sites['x'][site]), 
          'lon':(sites['y'][site])}               

    site_index = site_index = sites.iloc[site].name
    print(site_index)

    outfilename=f'{output_loc}{site_index}.csv'
    print (outfilename)


1
running for index 1
69
/g/data/r78/rjd547/Melbourne_water/69.csv
2
running for index 2
123
/g/data/r78/rjd547/Melbourne_water/123.csv
3
running for index 3
60
/g/data/r78/rjd547/Melbourne_water/60.csv
4
running for index 4
145
/g/data/r78/rjd547/Melbourne_water/145.csv
5
running for index 5
106
/g/data/r78/rjd547/Melbourne_water/106.csv
6
running for index 6
85
/g/data/r78/rjd547/Melbourne_water/85.csv
7
running for index 7
102
/g/data/r78/rjd547/Melbourne_water/102.csv
8
running for index 8
126
/g/data/r78/rjd547/Melbourne_water/126.csv
9
running for index 9
17
/g/data/r78/rjd547/Melbourne_water/17.csv
10
running for index 10
47
/g/data/r78/rjd547/Melbourne_water/47.csv
11
running for index 11
98
/g/data/r78/rjd547/Melbourne_water/98.csv
12
running for index 12
8
/g/data/r78/rjd547/Melbourne_water/8.csv
13
running for index 13
56
/g/data/r78/rjd547/Melbourne_water/56.csv
14
running for index 14
27
/g/data/r78/rjd547/Melbourne_water/27.csv
15
running for index 15
15
/g/data/r78/rjd54

KeyError: 206

In [ ]:
    ls578 = DEADataHandling.load_clearlandsat(dc=dc, query=query, product='nbart')

    ### Calculate NDVI 

    ### Calculate the tasseled cap indices

    #transform the nbart into tci - we want unthresholded, which is why we set drop_tci_bands to False
    tci = TasseledCapTools.thresholded_tasseled_cap(ls578,wetness_threshold=-350, drop=True , drop_tc_bands=False)

    tcw = tci.wetness
    #tcg = tci.greenness

#     ndvi = BandIndices.calculate_indices(ls578, index='NDVI')

#     ### create and fill pandas dataframe to write to csv

#     #drop dimensions of length 1
#     ndvi = ndvi.squeeze('x')
#     ndvi = ndvi.squeeze('y')

#     #make a new dataframe using the data from the xarray of ndvi
#     NDVI = pd.DataFrame(data=ndvi.data, index=ndvi.time,columns=['ndvi'])

#     #rename the dataframe so that we can add the other indexes
#     INDICIES=NDVI

    #again drop dims
    TCW = tcw.squeeze().data
    #TCG = tcg.squeeze().data

    # set up final dataframe to write to file 'INDICES'

    INDICIES['tcw']=TCW
   # INDICIES['tcg']=TCG
    #we have some values where the pixel is cloudy. Drop these values
    INDICES = INDICIES.dropna(axis='index', thresh =2)

    INDICES.to_csv(outfilename)

    !ls /g/data/r78/rjd547/LTIM_veg_monitoring/*.csv